In [9]:
import sys
sys.path.append('../')
from src.load_data import load_TEP_data_test
from src.load_data import load_TEP_data
from src.Sampler import *
from src.TEP_evaluation_Hotelling_func import *
import optuna
from optuna.samplers import TPESampler
from src.utils import dict_fault_ratio_fault,dict_fault_ratio_false_faults_found

normalised=False

model_name="Hotelling"


if normalised:
    list_features_group=[[19, 45],
     [9, 46],
     [1, 2, 3, 4, 5, 7, 10, 13, 20, 21, 41, 42, 44],
     [8, 50],
     [17, 18, 49],
     [0, 43],
     [11, 47],
     [16, 51],
     [22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35],
     [6, 12, 15],
     [14, 48],
     [36, 37, 38, 39, 40]]



else:
     list_features_group = [
[0, 43],
[1, 2, 3, 4, 5, 7, 8, 9, 10, 13, 19, 20, 21, 22, 23, 24,
 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38,
 39, 40, 41, 42, 44, 46, 50],
[14, 16, 48, 51],
[6, 12, 15, 45],
[11, 47],
[17, 18, 49]
]



Training_df,Testing_df,Validation_df,cols_feature=load_TEP_data()
parameters_trained_hotelling=train_hotelling_model(Training_df,cols_feature,list_features_group)



# Define a function to optimize
def objective(trial):
    
    
    value_to_obtain=1e-3
    # Define the hyperparameters to optimize
    q = trial.suggest_uniform('q', 0, 1)
    
    # Call the function to optimize
    output = generate_validation_results(parameters_trained_hotelling, Training_df, Validation_df, cols_feature, list_features_group, q)
    
    # Calculate the difference between the output and the value to obtain
    diff = np.abs(output - value_to_obtain)
    
    return diff

# Define the study
study = optuna.create_study(direction='minimize')

# Optimize the function
study.optimize(objective, n_trials=30)

# Get the best hyperparameters
best_params = study.best_params
best_q = best_params['q']

# Call the function with the best hyperparameters
results=generate_test_results(parameters_trained_hotelling, Training_df, Testing_df, cols_feature, list_features_group, best_q)


dict_fault=dict_fault_ratio_fault(results[1])
df_fault = pd.DataFrame.from_dict(dict_fault, orient='index', columns=['Ratio_faults_found'])
df_fault.to_csv('../Results/TEP/Ratio_faults_found_df_TEP_'+str(model_name)+"_"+str(normalised)+'.csv')
dict_fault=dict_fault_ratio_false_faults_found(results[1])
df_ratio_false_alarms = pd.DataFrame.from_dict(dict_fault, orient='index', columns=['Ratio_false_alarms'])
df_ratio_false_alarms.to_csv('../Results/TEP/Ratio_false_alarms_df_TEP_'+str(model_name)+"_"+str(normalised)+'.csv')

import pickle
# create a list

# define the file name for the pickle file
file_name = '../Results/TEP/Global_res_TEP_'+str(model_name)+"_"+str(normalised)+'.pkl'

# open the file in write binary mode
with open(file_name, "wb") as f:
    # dump the list to the file using pickle
    pickle.dump(results, f)

[I 2023-05-09 07:36:48,532] A new study created in memory with name: no-name-9f446449-943a-4cf8-ac4c-737bbdcd9b2f
/tmp/ipykernel_10516/3239429059.py:62: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  q = trial.suggest_uniform('q', 0, 1)
[I 2023-05-09 07:36:48,936] Trial 0 finished with value: 0.83772 and parameters: {'q': 0.83465845948639}. Best is trial 0 with value: 0.83772.
/tmp/ipykernel_10516/3239429059.py:62: FutureWarning: suggest_uniform has been deprecated in v3.0.0. This feature will be removed in v6.0.0. See https://github.com/optuna/optuna/releases/tag/v3.0.0. Use :func:`~optuna.trial.Trial.suggest_float` instead.
  q = trial.suggest_uniform('q', 0, 1)
[I 2023-05-09 07:36:49,311] Trial 1 finished with value: 0.3 and parameters: {'q': 0.29326412384734}. Best is trial 1 with value: 0.3.
/tmp/ipykernel_10516/3